In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import tensorflow as tf
import numpy as np
from tqdm import tqdm, trange
import os, sys
sys.path.append(os.getcwd())

Populating the interactive namespace from numpy and matplotlib


In [2]:
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from helper.recognition.resnet50 import *
from helper.recognition.efficientnet import *
from helper.recognition.biLSTM import *
from helper.recognition.crnn import ctc_lambda_fn

In [3]:
inputs = Input(shape=(299,299,3), name='inputs')

In [4]:
effnet = efficientnetB3(input_tensor=inputs)

In [5]:
eff_out = effnet.output
eff_out

<tf.Tensor 'top_activation/Identity:0' shape=(None, 10, 10, 1536) dtype=float32>

In [6]:
inner = Reshape(target_shape=(int(eff_out.shape[1]),int(eff_out.shape[2]*eff_out.shape[3])), name=f'reshape_connect')(eff_out)
inner

<tf.Tensor 'reshape_connect/Identity:0' shape=(None, 10, 15360) dtype=float32>

In [7]:
inner = Dense(64, activation='relu',kernel_initializer='he_normal', name=f'dense_connect')(inner)
inner

<tf.Tensor 'dense_connect/Identity:0' shape=(None, 10, 64) dtype=float32>

In [8]:
y_pred = biLSTM(inner, 256)
y_pred

<tf.Tensor 'softmax/Identity:0' shape=(None, 10, 1000) dtype=float32>

In [9]:
labels = Input(name='label', shape=[16, ], dtype='float32')
input_length = Input(name='input_length', shape=[1, ], dtype='int64')
label_length = Input(name='label_length', shape=[1, ], dtype='int64')
outputs = Lambda(ctc_lambda_fn, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])
outputs

<tf.Tensor 'ctc/Identity:0' shape=(None, 1) dtype=float32>

In [10]:
train = Model(inputs=[inputs, labels, input_length, label_length], outputs=outputs, name=f'train_crnn_efficientnet')
train.summary()

Model: "train_crnn_efficientnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 301, 301, 3)  0           inputs[0][0]                     
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 150, 150, 40) 1080        stem_conv_pad[0][0]              
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 150, 150, 40) 160         stem_conv[0][0]                  
____________________________________________________________________________